In [1]:
from sympy.solvers import solve
from sympy.vector import *
from sympy import *
import numpy as np
import transforms3d

init_printing()

# Definition des vecteurs

In [2]:
B_NED     = eye(3)                               # Base dans le repère NED
B_B       = eye(3)                               # Base dans le repère body


Omega     = MatrixSymbol('Omega',3,1)                  # Vecteur de rotation
R         = MatrixSymbol('R', 3, 3)                    # Matrice de rotation
v_B        = MatrixSymbol('v_B',3,1)                    # Vitesse du corps (repère drone)
v_W        = MatrixSymbol('v_W',3,1)                    # Vitesse du vent (repère NED)

delta_rotor     = MatrixSymbol('delta_{rotor}', 3,1 )              # Position du corps dans le repère NED


X_NED     = MatrixSymbol('X_{NED}', 3,1 )              # Position du corps dans le repère NED
c_p        = MatrixSymbol('c_p', 3,1)                   # Centre de poussé
A         = Symbol('A', real=True)                     # 0.5 * rho * S 
Air_speed_earth  = MatrixSymbol('V_{air_{NED}}', 3, 1) # air speed dans le repère NED
Air_speed_body   = MatrixSymbol('V_{air_{body}}', 3, 1)# air speed dans le repère body

omega,ct=symbols('omega,C_t',real=True)                # vitesse de rotation des moteurs, et coefficient de poussé des helices
c_d,c_l=symbols('C_d,C_l',real=True)                # vitesse de rotation des moteurs, et coefficient de poussé des helices

nb_rotor = 4

## Définition des vecteurs des forces aérodynamiques dans la base NED

In [3]:
frontward_B = B_B.col(0)
upward_B    = B_B.col(2)
crossward_B = B_B.col(1)

frontward_NED = B_NED.col(0)
upward_NED    = B_NED.col(2)
crossward_NED = B_NED.col(1)


Fonction permettant de calculer les forces aéro en fonction de la vitesse Air, de la vitesse de déplacement et des coeffs aéro dans le repère body

In [4]:
def GenForceWing(Omega, A, cp, vB, vW, R, crossward_NED, Cd, Cl, VelLim=0.1):

    Air_speed_earth = MatAdd(vB, -vW)
    Air_speed_body  = (R.T* Air_speed_earth)
    Air_speed_body  = Air_speed_body - cross(Omega,cp)
    
    Air_speed_earth_norm = 0
    for i in Air_speed_earth : 
        Air_speed_earth_norm = Air_speed_earth_norm + i**2
    Air_speed_earth_norm = 1#Air_speed_earth_norm**0.5
    VelinLDPlane    = MatAdd(Air_speed_earth, - MatMul(MatMul(Air_speed_earth, crossward_NED.T) , Air_speed_earth/ Air_speed_earth_norm))
    
    VelinLDPlane_norm = 0
    for i in VelinLDPlane : 
        VelinLDPlane_norm = VelinLDPlane_norm + i**2
    VelinLDPlane_norm = 1#VelinLDPlane_norm**0.5
    
    dragDirection = -VelinLDPlane/VelinLDPlane_norm # if VelinLDPlane_norm > VelLim else Matrix([0,0,0])
    liftDirection = (crossward_NED).cross(dragDirection) # if crossward_NED.norm() > VelLim else Matrix([0,0,0])

    L = A * VelinLDPlane_norm**2 * dragDirection * Cl
    D = A * VelinLDPlane_norm**2 * liftDirection * Cd

    F_wing = L+D 

    Torque_wing =  cross(cp,((R.T*F_wing)))

    return F_wing, Torque_wing

In [5]:
Omega_test = Matrix([[1,0,0],[0,1,0],[0,0,1]])           # Matrice Omega 
vB = Matrix([1,0,0])                                     # Vitesse dans le repère body
vW = Matrix([0, 0,1])                                   # Vitesse du vent dans le repère NED
cp = Matrix([0,-1,0])
#R = Matrix([[0,1,0],[-1,0,0],[0,0,1]])
Cd =0.1
Cl =1
Aire = 0.5 * 1.292 * 0.1 *0.5 

F1,Q1 = GenForceWing(Omega, A, c_p, v_B, v_W, R, crossward_NED, c_d, c_l, VelLim=0.1)
pprint(F1)
#pprint(Q1)

                                                          ⎡A⋅C_d⋅((v_B_10 - v_
                                                          ⎢                   
-A⋅Cₗ⋅(-v_W -(-v_W + v_B)⋅[0  1  0]⋅(-v_W + v_B) + v_B) + ⎢                   
                                                          ⎢                   
                                                          ⎣A⋅C_d⋅(-(v_B_00 - v

W_10)⋅(v_B_20 - v_W_20) - v_B_20 + v_W_20) ⎤
                                           ⎥
           0                               ⎥
                                           ⎥
_W_00)⋅(v_B_10 - v_W_10) + v_B_00 - v_W_00)⎦


Lambdyfication des fonction des forces des ailes 

In [6]:
F_function = lambdify((Omega, A, c_p, v_B, v_W, R, crossward_NED, c_d, c_l), F1)

SyntaxError: cannot assign to literal (<lambdifygenerated-1>, line 2)

Fonction permettant de calculer la poussé des moteurs

In [ ]:
def GenForceMoteur(Omega, ct, cq, omega_rotor, cp, Air_speed_body, motor_axis_in_body_frame):
    
    air_speed_in_rotor_frame = Air_speed_body - Omega * cp
    Axial_speed = air_speed_in_rotor_frame.cross(motor_axis_in_body_frame)
    lat_speed = air_speed_in_rotor_frame - Axial_speed.cross(motor_axis_in_body_frame)
        
    T = np.clip(0, ct*omega_rotor**2 * np.clip(1.0, ct*air_speed_in_rotor_frame.norm(),0.1), 1e5)
    
    T_vec = T * motor_axis_in_body_frame 
    
    torque = - omega_rotor * cq * lat_speed
    torque = - cq * T * motor_axis_in_body_frame 
    torque = torque.T
    torque_at_body_center = torque.T + (cp).cross(T_vec.T)
        
    return T_vec, torque_at_body_center

In [ ]:
Omega_rotor = 2000*60/2/np.pi
Air_speed_body = Matrix([10,1,0])
motor_axis_in_body_frame = Matrix([1,0,0])
ct =1.1e-4
cq =1e-6
F1,Q1 = GenForceMoteur(Omega_test, ct, cq, Omega_rotor, cp , Air_speed_body, motor_axis_in_body_frame)
pprint(F1)
pprint(Q1)